In [1]:
#%pip install pybamm -q    # install PyBaMM if it is not installed
import os
import matplotlib.pyplot as plt
import numpy as np
import pybamm
import pandas as pd
import timeit
from matplotlib import style
style.use('ggplot')
start = timeit.default_timer()
os.chdir(pybamm.__path__[0]+'/..')
pybamm.set_logging_level("INFO")

In [2]:
model = pybamm.lithium_ion.BasicDFN()
chemistry = pybamm.parameter_sets.Chen2020
param = pybamm.ParameterValues(chemistry=chemistry)

# model = pybamm.lithium_ion.DFN()
# model = pybamm.lithium_ion.BasicDFN()
# chemistry = pybamm.parameter_sets.Chen2020
# param = pybamm.ParameterValues(chemistry=chemistry)

param.update({"Upper voltage cut-off [V]": 4.5})
param.update({"Lower voltage cut-off [V]": 2.5})
param.update({
#    "Maximum concentration in negative electrode of phase 1 [mol.m-3]":28700,
#    "Initial concentration in negative electrode of phase 1 [mol.m-3]":23000,
    "Negative electrode mass fraction of phase 1":1,
#    "Negative electrode diffusivity of phase 1 [m2.s-1]":3.3E-14,
    "Negative electrode mass fraction of phase 2":0,
    "Initial concentration in negative electrode of phase 2 [mol.m-3]":270000,
    "Maximum concentration in negative electrode of phase 2 [mol.m-3]":278000,
#     #
     "Maximum concentration in negative electrode of phase 1 [mol.m-3]":33133,
     "Initial concentration in negative electrode of phase 1 [mol.m-3]":29866,
#     "Negative electrode mass fraction of phase 1":1,
#     "Negative electrode diffusivity of phase 1 [m2.s-1]":3.3E-14,
#     "Negative electrode mass fraction of phase 2":0.00,
#     "Initial concentration in negative electrode of phase 2 [mol.m-3]":275220,
#     #
    })
param05 = param
param1 = param
param15 = param

# initial concentration of graphite 29866 [mol.m-3]
# -> 0.934 * 25830 + 0.0866 * 275220

C_rates = 0.5
Q_cell = 4.81 # [Ah]
period = 1 / C_rates * Q_cell / 5
transition = period / 20
timestep=np.array([0.1, period, 2]) * 3600
points=[41, 201, 201]
t_evals=[]
t0 = 0
for i in range(0,len(timestep)):
    t1 = np.linspace(t0, t0 + timestep[i] - transition, points[i])
    t0 += timestep[i]
    if i < len(timestep)-1:
        t2 = np.linspace(t0 - transition, t0 + transition, 41) # transition points
    else:
        t2 = []
    t_evals=np.concatenate((t_evals, t1 ,t2))

t_evals = np.unique(t_evals)
capacity = param["Nominal cell capacity [A.h]"]
I_load = C_rates * capacity  

def I_fun(A):
    def current(t):
        C_rates = 0.5
        Q_cell = 4.81 # [Ah]
        period = 1 / C_rates * Q_cell / 5
        timestep=np.array([0.1, period, 2]) * 3600
        k = 10
        rest = pybamm.sigmoid( t , timestep[0], k)
        discharge = pybamm.sigmoid( t , timestep[0] + timestep[1], k) - rest
        return A * discharge
        # return A * (t < period) - A * (t >= period)
        # return A * 0.2 * pybamm.sin(2 * np.pi * t / 7200 )
    return current

param05["Current function [A]"] = I_fun(I_load)

sim1 = pybamm.Simulation(
    model,
    parameter_values=param05,
    solver=pybamm.CasadiSolver(dt_max = 10),
)
solution05 = sim1.solve(t_eval = t_evals)
stop = timeit.default_timer()
print("running time: " + str(stop - start) + "s")

KeyError: "Cannot update parameter 'Negative electrode mass fraction of phase 1' as it does not have a default value. ('Negative electrode mass fraction of phase 1' not found. Best matches are ['Negative electrode electrons in reaction', 'Negative electrode active material volume fraction', 'Negative electrode thickness [m]']). If you are sure you want to update this parameter, use param.update({{name: value}}, check_already_exists=False)"

In [ ]:
## for 1 C discharge
C_rates = 1
Q_cell = 4.81 # [Ah]
period = 1 / C_rates * Q_cell / 5
transition = period / 20
timestep=np.array([0.1, period, 2]) * 3600
points=[41, 201, 201]
t_evals=[]
t0 = 0
for i in range(0,len(timestep)):
    t1 = np.linspace(t0, t0 + timestep[i] - transition, points[i])
    t0 += timestep[i]
    if i < len(timestep)-1:
        t2 = np.linspace(t0 - transition, t0 + transition, 41) # transition points
    else:
        t2 = []
    t_evals=np.concatenate((t_evals, t1 ,t2))

t_evals = np.unique(t_evals)
capacity = param["Nominal cell capacity [A.h]"]
I_load = C_rates * capacity  

def I_fun(A):
    def current(t):
        C_rates = 1
        Q_cell = 4.81 # [Ah]
        period = 1 / C_rates * Q_cell / 5
        timestep=np.array([0.1, period, 2]) * 3600
        k = 5
        rest = pybamm.sigmoid( t , timestep[0], k)
        discharge = pybamm.sigmoid( t , timestep[0] + timestep[1], k) - rest
        return A * discharge
        # return A * (t < period) - A * (t >= period)
        # return A * 0.2 * pybamm.sin(2 * np.pi * t / 7200 )
    return current

param1["Current function [A]"] = I_fun(I_load)

sim1 = pybamm.Simulation(
    model,
    parameter_values=param1,
    solver=pybamm.CasadiSolver(dt_max = 10),
)
solution1 = sim1.solve(t_eval = t_evals)
stop1 = timeit.default_timer()
print("running time: " + str(stop1 - stop) + "s")

In [ ]:
## for 1.5 C charge
C_rates = 1.5
Q_cell = 4.8 # [Ah]
period = 1 / C_rates * Q_cell / 5
transition = period / 20
timestep=np.array([0.1, period, 2]) * 3600
points=[41, 201, 201]
t_evals=[]
t0 = 0
for i in range(0,len(timestep)):
    t1 = np.linspace(t0, t0 + timestep[i] - transition, points[i])
    t0 += timestep[i]
    if i < len(timestep)-1:
        t2 = np.linspace(t0 - transition, t0 + transition, 41) # transition points
    else:
        t2 = []
    t_evals=np.concatenate((t_evals, t1 ,t2))

t_evals = np.unique(t_evals)
capacity = param["Nominal cell capacity [A.h]"]
I_load = C_rates * capacity  

def I_fun(A):
    def current(t):
        C_rates = 1.5
        Q_cell = 4.8 # [Ah]
        period = 1 / C_rates * Q_cell / 5
        timestep=np.array([0.1, period, 2]) * 3600
        k = 5
        rest = pybamm.sigmoid( t , timestep[0], k)
        discharge = pybamm.sigmoid( t , timestep[0] + timestep[1], k) - rest
        return A * discharge
        # return A * (t < period) - A * (t >= period)
        # return A * 0.2 * pybamm.sin(2 * np.pi * t / 7200 )
    return current

param15["Current function [A]"] = I_fun(I_load)

sim1 = pybamm.Simulation(
    model,
    parameter_values=param15,
    solver=pybamm.CasadiSolver(dt_max = 10),
)
solution1_5 = sim1.solve(t_eval = t_evals)
stop2 = timeit.default_timer()
print("running time: " + str(stop2 - stop1) + "s")

In [ ]:
# plot
plot = pybamm.QuickPlot( solution05, [       
        "Current [A]",
        "Terminal voltage [V]",
        "X-averaged negative electrode open circuit potential of phase 1 [V]",
        "X-averaged negative electrode open circuit potential of phase 2 [V]",
    ]
)
plot.dynamic_plot()


In [ ]:
path = "wip/data/chen/"
V_0_5C_exp=pd.read_csv (path + "0.5C_exp.csv", delimiter= ',',header=None)
V_0_5C_sim=pd.read_csv (path + "0.5C_sim.csv", delimiter= ',',header=None)
V_1C_exp=pd.read_csv (path + "1C_exp.csv", delimiter= ',',header=None)
V_1C_sim=pd.read_csv (path + "1C_sim.csv", delimiter= ',',header=None)
V_1_5C_exp=pd.read_csv (path + "1.5C_exp.csv", delimiter= ',',header=None)
V_1_5C_sim=pd.read_csv (path + "1.5C_sim.csv", delimiter= ',',header=None)

t_all0_5C = solution05["Time [s]"].entries / 3600
V_n0_5C = solution05["Terminal voltage [V]"].entries
t_all1C = solution1["Time [s]"].entries / 3600
V_n1C = solution1["Terminal voltage [V]"].entries
t_all1_5C = solution1_5["Time [s]"].entries / 3600
V_n1_5C = solution1_5["Terminal voltage [V]"].entries

In [ ]:
f, (ax1, ax2, ax3) = plt.subplots(1, 3 ,figsize=(14,4))
ax1.plot(V_0_5C_exp.values[:,0], V_0_5C_exp.values[:,1],'k-',label="Experiment")
ax1.plot(V_0_5C_sim.values[:,0], V_0_5C_sim.values[:,1],'r:',label="Simulation (Chen et al. 2020)")
ax1.plot(t_all0_5C - 0.1, V_n0_5C ,'b--',label="Simulation (Current)")
ax1.set_title('0.5 C')
ax1.set_xlabel('Time [h]')
ax1.set_ylabel('Terminal voltage [V]')
ax1.legend()

ax2.plot(V_1C_exp.values[:,0], V_1C_exp.values[:,1],'k-',label="Experiment")
ax2.plot(V_1C_sim.values[:,0], V_1C_sim.values[:,1],'r:',label="Simulation (Chen et al. 2020)")
ax2.plot(t_all1C - 0.1, V_n1C ,'b--',label="Simulation (Current)")
ax2.set_title('1 C')
ax2.set_xlabel('Time [h]')
ax2.set_ylabel('Terminal voltage [V]')
ax2.legend()

ax3.plot(V_1_5C_exp.values[:,0], V_1_5C_exp.values[:,1],'k-',label="Experiment")
ax3.plot(V_1_5C_sim.values[:,0], V_1_5C_sim.values[:,1],'r:',label="Experiment")
ax3.plot(t_all1_5C - 0.1, V_n1_5C ,'b--',label="Simulation (Current)")
ax3.set_title('1.5 C')
ax3.set_xlabel('Time [h]')
ax3.set_ylabel('Terminal voltage [V]')
ax3.legend()

In [ ]:
f, (ax1, ax2, ax3) = plt.subplots(1, 3 ,figsize=(14,4))
# ax1.plot(V_0_5C_exp.values[:,0], V_0_5C_exp.values[:,1],'k-',label="Experiment")
V_exp1 = np.interp(V_0_5C_exp.values[:,0], V_0_5C_sim.values[:,0], V_0_5C_sim.values[:,1])
ax1.plot(V_0_5C_exp.values[:,0], abs(V_0_5C_exp.values[:,1] - V_exp1),'r:',label="Simulation (Chen et al. 2020)")
V_exp2 = np.interp(V_0_5C_exp.values[:,0],t_all0_5C - 0.1, V_n0_5C)
ax1.plot(V_0_5C_exp.values[:,0], abs(V_0_5C_exp.values[:,1] - V_exp2) ,'b--',label="Simulation (Current)")
ax1.set_title('0.5 C')
ax1.set_xlabel('Time [h]')
ax1.set_ylabel('Voltage error [V]')
ax1.legend()
RMSE_0_5C = np.sqrt(((V_0_5C_exp.values[:,1] - V_exp2) ** 2 ).mean())
RMSE_0_5C_chen = np.sqrt(((V_0_5C_exp.values[:,1] - V_exp1) ** 2 ).mean())

#ax2.plot(V_1C_exp.values[:,0], V_1C_exp.values[:,1],'k-',label="Experiment")
V_exp1 = np.interp(V_1C_exp.values[:,0], V_1C_sim.values[:,0], V_1C_sim.values[:,1])
ax2.plot(V_1C_exp.values[:,0], abs(V_1C_exp.values[:,1] - V_exp1),'r:',label="Simulation (Chen et al. 2020)")
V_exp2 = np.interp(V_1C_exp.values[:,0], t_all1C - 0.1, V_n1C)
ax2.plot(V_1C_exp.values[:,0], abs(V_1C_exp.values[:,1] - V_exp2),'b--',label="Simulation (Current)")
ax2.set_title('1 C')
ax2.set_xlabel('Time [h]')
ax2.set_ylabel('Terminal voltage [V]')
ax2.legend()
RMSE_1C = np.sqrt(((V_1C_exp.values[:,1] - V_exp2) ** 2 ).mean())
RMSE_1C_chen = np.sqrt(((V_1C_exp.values[:,1] - V_exp1) ** 2 ).mean())

#ax3.plot(V_1_5C_exp.values[:,0], V_1_5C_exp.values[:,1],'k-',label="Experiment")
V_exp1 = np.interp(V_1_5C_exp.values[:,0], V_1_5C_sim.values[:,0], V_1_5C_sim.values[:,1])
ax3.plot(V_1_5C_exp.values[:,0], abs(V_1_5C_exp.values[:,1] - V_exp1),'r:',label="Simulation (Chen et al. 2020)")
V_exp2 = np.interp(V_1_5C_exp.values[:,0], t_all1_5C - 0.1, V_n1_5C)
ax3.plot(V_1_5C_exp.values[:,0], abs(V_1_5C_exp.values[:,1] - V_exp2) ,'b--',label="Simulation (Current)")
ax3.set_title('1.5 C')
ax3.set_xlabel('Time [h]')
ax3.set_ylabel('Terminal voltage [V]')
ax3.legend()
RMSE_1_5C = np.sqrt(((V_1_5C_exp.values[:,1] - V_exp2) ** 2 ).mean())
RMSE_1_5C_chen = np.sqrt(((V_1_5C_exp.values[:,1] - V_exp1) ** 2 ).mean())
print([RMSE_0_5C, RMSE_1C, RMSE_1_5C])
print([RMSE_0_5C_chen, RMSE_1C_chen, RMSE_1_5C_chen])

In [ ]:
return

In [ ]:
times = solution1_5["Time [s]"].entries / 3600
c_s_xrav_n_p1 = solution1_5["Averaged negative electrode concentration of phase 1"].entries
c_s_xrav_n_p2 = solution1_5["Averaged negative electrode concentration of phase 2"].entries
c_s_xrav_p = solution1_5["Averaged positive electrode concentration"].entries
plt.plot(times, c_s_xrav_n_p1,'r-', label='Graphite')
plt.plot(times, c_s_xrav_n_p2, 'b:',label='Silicon')
plt.plot(times, c_s_xrav_p, 'g--',label='NMC')
plt.xlabel("Time [h]")
plt.ylabel("$c/c_\mathrm{max}$")
# plt.ylim([0,0.02])
plt.legend()

In [ ]:
times = solution1_5["Time [s]"].entries / 3600
ocp_p1 = solution1_5["X-averaged negative electrode open circuit potential of phase 1 [V]"].entries
ocp_p2 = solution1_5["X-averaged negative electrode open circuit potential of phase 2 [V]"].entries
plt.plot(times,ocp_p1,'r--',label='Graphite')
plt.plot(times,ocp_p2,'b:',label='Silicon')
plt.xlabel("Time [h]")
plt.ylabel("Open circuit potential [V]")
plt.title("Cell open circuit potential [V]")
plt.legend()
# plt.xlim([100, 103])
# plt.ylim([0.4, 1])

In [ ]:
j_n_p1_av = solution1_5["X-averaged negative electrode interfacial current density of phase 1 [A.m-2]"].entries
j_n_p2_av = solution1_5["X-averaged negative electrode interfacial current density of phase 2 [A.m-2]"].entries
plt.plot(times, j_n_p1_av,'r-', label='Graphite')
plt.plot(times, j_n_p2_av, 'b:',label='Silicon')
plt.xlabel("Time [h]")
plt.ylabel("Interfacial current density [A/m$^{2}$]")
# plt.ylim([-0.2, 0.8])
plt.xlim([0.7, 0.8])
plt.legend()

In [ ]:
print(V_1_5C_exp.values[:,1] - V_exp1)